In [1]:
import cv2 as cv
import numpy as np
import os

# Ruta del dataset
dataSet = 'C://Users//ASUS//Desktop//universidad//IA//emociones//Emociones'
faces = os.listdir(dataSet)
print(faces)

labels = []
facesData = []
label = 0

# Cargar las imágenes del dataset
for face in faces:
    facePath = dataSet + '/' + face
    for faceName in os.listdir(facePath):
        labels.append(label)
        facesData.append(cv.imread(facePath + '/' + faceName, 0))
    label += 1

# Crear el reconocedor LBPH y entrenarlo
faceRecognizer = cv.face.LBPHFaceRecognizer_create()
faceRecognizer.train(facesData, np.array(labels))
faceRecognizer.write('carasLBPHFace.xml')

print("Modelo entrenado y guardado como 'imgLBPHFace.xml'")


['Feliz', 'Neutral', 'Sorpresa', 'Triste']
Modelo entrenado y guardado como 'imgLBPHFace.xml'


In [3]:
import cv2 as cv
import os

# Leer el modelo entrenado
faceRecognizer = cv.face.LBPHFaceRecognizer_create()
faceRecognizer.read('carasLBPHFace.xml')

# Iniciar captura de video
cap = cv.VideoCapture(0)
rostro = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Voltear el frame horizontalmente para modo espejo
    frame = cv.flip(frame, 1)
    
    # Convertir a escala de grises y aplicar ecualización del histograma
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    gray = cv.equalizeHist(gray)
    gray = cv.GaussianBlur(gray, (5, 5), 0)  # Suavizar la imagen
    cpGray = gray.copy()
    
    # Detección de rostros
    rostros = rostro.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)
    for (x, y, w, h) in rostros:
        frame2 = cpGray[y:y+h, x:x+w]
        frame2 = cv.resize(frame2, (100, 100), interpolation=cv.INTER_CUBIC)
        result = faceRecognizer.predict(frame2)
        
        # Si la predicción es confiable, mostrar la etiqueta y el rectángulo
        if result[1] < 2800:
            cv.putText(frame, '{}'.format(faces[result[0]]), (x, y-25), 2, 1.1, (0, 255, 0), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    # Mostrar el frame
    cv.imshow('frame', frame)
    k = cv.waitKey(1)
    if k == 27:
        break

# Liberar recursos
cap.release()
cv.destroyAllWindows()
